In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, njit
from general_functions import *

In [11]:
@njit
def difference_numba(phi_init, phi_final, N, S, M, normalised=False):
    
    if normalised: 
        return np.sum(np.abs(phi_final - phi_init)) / (N*(S-1)*M)
    else: 
        return np.sum(np.abs(phi_final - phi_init))

@njit
def proba_diag_grammar(N, S, M, lambd, max_steps=100000, n_runs=10, phi_init=None, max_diff=1):

    final_diag_counter = 0
    diffs = []

    phi_optimal = np.zeros((N,S,M))*1/S
    for s in range(S):
        for m in range(M):
            if s == m: phi_optimal[:, s, m] = 1

    for run in range(n_runs):

        phi = np.ones((N, S, M)) / S if phi_init is None else phi_init

        for step in range(max_steps):

            intend = np.random.randint(M) # meaning intended -- rho = 1/M, uniform distribution
            speaker = np.random.randint(N)
            listener = np.random.randint(N)
            while listener == speaker: listener = np.random.randint(N)

            produce = rand_choice_numba(prob = phi[speaker,:,intend])
            infer = rand_choice_numba(prob = phi[listener,produce,:] / np.sum(phi[listener,produce,:]))
            # updating phi
            feedback = lambd if infer == intend else - lambd
            phi[speaker,produce,intend] += (feedback * phi[speaker,produce,intend] * (1 - phi[speaker,produce,intend]))
            phi[speaker,:,intend] /= np.sum(phi[speaker,:,intend]) # normalise along signal axis

            if step % 200 == 0 and step > 0:

                if equilibrium_numba(phi, N, S, M): 
                    break
        
        diff = difference_numba(phi_init=phi_optimal, phi_final=phi, N=N, S=S, M=M)
        # print(diff)
        # print(phi)
        diffs.append(diff)

        if diff < max_diff: final_diag_counter += 1
    
    print(diffs)

    return final_diag_counter / n_runs

def same_but_non_numba(N, S, M, lambd, max_steps=10000, n_runs=10, phi_init=None, max_diff=1):

    final_diag_counter = 0
    diffs = []

    mu_pos, mu_neg = lambd, -lambd

    for run in range(n_runs):

        phi = np.ones((N, S, M)) / S if phi_init is None else phi_init.copy()

        for step in range(max_steps):

            speaker, listener = choose_agents(N)
            intend = choose_meaning(M)
            signal = choose_signal(S, prob_array=phi[speaker,:,intend])
            infer = infer_meaning(M, prob_array=phi[listener,signal,:] / np.sum(phi[listener,signal,:]))
            feedback = get_feedback(mu_pos, mu_neg, intend, infer, speaker, M)

            phi[speaker,signal,intend] += feedback * U(phi[speaker,signal,intend])
            phi[speaker,:,intend] /= np.sum(phi[speaker,:,intend])

            if step % 200 == 0 and step > 0:

                if equilibrium(phi, N, S, M, thresh_frac=0.95): 
                    break
        
        diff = difference(phi_init=optimal_grammar(N, S, M), phi_final=phi, N=N, S=S, M=M)
        diffs.append(diff)

        if diff < max_diff: final_diag_counter += 1

    print(diffs)
    return final_diag_counter / n_runs

In [12]:
# code to find how strong of a perturbation is needed to permanently move the system out of equilibrium 
# and converge to one specific mapping
# to do that we look at the probability of endind in the same state as the initialised one, depending on how close to it we are

N, S, M, lambd = 2, 3, 3, 0.1
n_runs = 100

for diag_val in np.linspace(1/S, 1/S + 0.1, 5):

    phi_init = initiate_phi(N, S, M, method='diag_val', diag_val=diag_val)
    # print(phi_init)

    # p = proba_diag_grammar(N, S, M, lambd, n_runs=n_runs, phi_init=phi_init) numba doesn't work here
    p = same_but_non_numba(N, S, M, lambd, n_runs=n_runs, phi_init=phi_init)

    print(f'{diag_val:.3f}, p = {p:.5f}')



[11.830110813268456, 11.85152395275719, 11.862546345610141, 11.876724460560494, 11.887108219599513, 11.896876310069448, 11.903509272248334, 11.911693182693428, 11.917849156101926, 11.920939437137395, 11.927767005721394, 11.931264516341793, 11.934296854698426, 11.939406581395103, 11.941894547151225, 11.943559904625918, 11.945123985945019, 11.948268922732195, 11.952924805276274, 11.954034808168174, 11.955052540319242, 11.956717364885074, 11.957641312049487, 11.958486153100937, 11.959317527084877, 11.960127659266693, 11.961426714764709, 11.962879585530242, 11.963583105315024, 11.964217923313617, 11.965317410968236, 11.965931970864949, 11.966469243030852, 11.967593590504336, 11.968090997275318, 11.968579472836916, 11.96946170100459, 11.96992443212533, 11.970782071328163, 11.971212024944688, 11.97164261579616, 11.972068098302836, 11.972462822284355, 11.973466633353404, 11.974370592569976, 11.974704479212184, 11.975517441376303, 11.97581992277242, 11.976097018009638, 11.976720473409141, 11.9